#*A/B Testing – Evaluación del rediseño*#
1. Carga de datos
2. Definición y análisis de KPIs: 
    Tasa de finalización
    Tiempo medio por paso
    Tasa de error
3. Pruebas estadísticas
4. Conclusiones

*1. CARGA DE DATOS*

In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

df_ab = pd.read_csv("../Datos procesados/df_ab_test.csv")



In [120]:
df_ab.head()


,client_id,age_group,gender,tenure_months,num_accounts,balance,balance_log,logins_6m,calls_6m,experiment_group,web_events,web_visits,web_steps,web_confirms
0,836976,60.5,Unknown,73.0,2.0,45105.30,10.716777,9.0,6.0,Test,11,2,5,3
1,2304905,58.0,Unknown,94.0,2.0,110860.30,11.616035,9.0,6.0,Control,6,1,5,1
2,1439522,32.0,Unknown,64.0,2.0,52467.79,10.867974,9.0,6.0,Test,5,2,4,0
3,1562045,49.0,Male,198.0,2.0,67454.65,11.119226,6.0,3.0,Test,1,1,1,0
4,5126305,33.0,Female,145.0,2.0,103671.75,11.548995,3.0,0.0,Control,1,1,1,0


*2. DEFINICIÓN DE KPI's*

 *2.1. Tasa de finalización. Proporción de usuarios que llegan al paso final "confirm" respecto al total de usuarios del grupo.*

In [121]:
# Total usuarios por grupo
df_ab.groupby("experiment_group")["client_id"].nunique()


experiment_group
Control    23532
Test       26968
Name: client_id, dtype: int64

In [122]:
# Usuarios que confirmaron
confirm_users = df_ab[df_ab["web_confirms"] > 0]

confirm_users.groupby("experiment_group")["client_id"].nunique()


experiment_group
Control    15434
Test       18687
Name: client_id, dtype: int64

El grupo Test tiene más usuarios que llegan a confirmación, en comparación con los de Control

In [123]:
conversion = (
    confirm_users.groupby("experiment_group")["client_id"].nunique()
    /
    df_ab.groupby("experiment_group")["client_id"].nunique()
)

conversion_pct = (conversion * 100).round(2)
conversion_pct



experiment_group
Control    65.59
Test       69.29
Name: client_id, dtype: float64

En virtud de este primer análisis, vemos que el nuevo diseño presenta una tasa de conversión del 69.29%, frente al 65.59% del diseño anterior, lo que supone una mejora de +3.70 puntos porcentuales.

*2.2 — Media de pasos para llegar a la confirmación. ¿El nuevo diseño hace que los usuarios tarden menos (o más) en completar el proceso?*

In [124]:
kpi2_steps = (
    df_ab
    .groupby("experiment_group")["web_steps"]
    .mean()
    .round(2)
)

kpi2_steps


experiment_group
Control    4.04
Test       4.18
Name: web_steps, dtype: float64

El nuevo diseño no reduce significativamente el número promedio de pasos necesarios para completar el proceso. La diferencia observada es mínima, de apenas 0.14 pasos, por lo que no podemos afirmar que exista una mejora en eficiencia.
El rediseño probablemente mejora claridad o experiencia, pero no simplifica el flujo estructuralmente.

*2.3. Tasa de error. Usuarios que realizan muchos pasos o, directamente, no llegan a confirmar.*


In [125]:

error_users = df_ab[
    (df_ab["web_steps"] > 0) & 
    (df_ab["web_confirms"] == 0)
]

error_rate = (
    error_users.groupby("experiment_group")["client_id"].nunique()
    /
    df_ab.groupby("experiment_group")["client_id"].nunique()
).round(4)


(error_rate * 100).round(2)



experiment_group
Control    34.41
Test       30.71
Name: client_id, dtype: float64

Con el nuevo diseño se observa una mejora aproximada del 10% (−3.7 puntos porcentuales) en la tasa de error, pasando del 34.41% en el diseño anterior al 30.71% en la versión bajo estudio.
Aunque se aprecia una reducción del abandono, una proporción relevante de usuarios sigue sin completar el proceso, lo que sugiere margen de mejora en la experiencia.

*3. ANÁLISIS ESTADÍSTICO*


3.1. Tasa de finalización. ¿Es la diferencia entre ambos grupos estadísticamente significativa?

#Hipótesis:
#H0: La tasa de conversión es igual en Control y Test
#H1: La tasa de conversión es diferente

In [126]:
from statsmodels.stats.proportion import proportions_ztest

# Número de conversiones
conversions = confirm_users.groupby("experiment_group")["client_id"].nunique()

# Total de usuarios por grupo
totals = df_ab.groupby("experiment_group")["client_id"].nunique()

# Orden consistente
conv = [conversions["Control"], conversions["Test"]]
nobs = [totals["Control"], totals["Test"]]

z_stat, p_value = proportions_ztest(count=conv, nobs=nobs)

z_stat, p_value


(np.float64(-8.8745141890702), np.float64(7.023933247581433e-19))

In [127]:
#3.2. Media de pasos para llegar a la confirmación. ¿Es la diferencia entre ambos grupos estadísticamente significativa?

#Hipótesis:
#H0: La media de pasos es igual en Control y Test
#H1: La media de pasos es diferente

from scipy.stats import ttest_ind

control_steps = df_ab[df_ab["experiment_group"] == "Control"]["web_steps"]
test_steps = df_ab[df_ab["experiment_group"] == "Test"]["web_steps"]

t_stat, p_value_steps = ttest_ind(control_steps, test_steps, equal_var=False)

t_stat, p_value_steps


(np.float64(-11.09475948070888), np.float64(1.4405840570768927e-28))

In [128]:
#3.3 Tasa de error. ¿Es la diferencia entre ambos grupos estadísticamente significativa?


#Hipótesis:
#H0: La tasa de error es igual en Control y Test
#H1: La tasa de error es diferente


In [129]:
# Usuarios con error
errors = error_users.groupby("experiment_group")["client_id"].nunique()

err = [errors["Control"], errors["Test"]]

z_stat_err, p_value_err = proportions_ztest(count=err, nobs=nobs)

z_stat_err, p_value_err


(np.float64(8.8745141890702), np.float64(7.023933247581433e-19))

En virtud de lo analizado, se considera:
1. El nuevo diseño mejora significativamente la tasa de conversión.
    Los usuarios expuestos a la versión Test presentan una conversión del 69.29%, frente al 65.59% del grupo Control.
    El test de proporciones arroja un p-value < 0.05, por lo que se rechaza la hipótesis nula de igualdad de conversión entre grupos.

2. La media de pasos difiere significativamente entre versiones.
    El grupo Test presenta un promedio ligeramente mayor de pasos (4.18 vs 4.04).
    Si bien la diferencia es estadísticamente significativa, el incremento es reducido y no necesariamente implica un deterioro en la experiencia, pudiendo responder a un flujo más detallado o estructurado.

3. La tasa de error se reduce de forma significativa en el nuevo diseño.
    El grupo Test presenta una tasa de error del 30.71%, frente al 34.41% del Control.
    El contraste de proporciones confirma que esta diferencia es estadísticamente significativa.

*4. CONCLUSIONES Y RECOMENDACIONES*

Luego de los análisis y estudios realizados, concluimos en:
1. El nuevo diseño mejora significativamente el desempeño general del proceso: incrementa la tasa de conversión (69.29% vs 65.59%) y reduce de forma significativa la tasa de error. Aunque el número medio de pasos aumenta levemente, este incremento no impacta negativamente en la conversión, lo que sugiere un flujo más claro y efectivo para los usuarios.

2. La combinación de mayor conversión y menor tasa de error indica una mejora en la calidad del recorrido y una reducción de fricciones críticas en la experiencia de usuario.

3. No obstante, el incremento absoluto en la tasa de finalización es de aproximadamente 3.7 puntos porcentuales, cifra inferior al umbral de costo-efectividad del 5% establecido por el negocio. Si bien la mejora es estadísticamente significativa, podría no ser suficiente para justificar su implementación definitiva únicamente desde el punto de vista económico.

4. En consecuencia, recomendamos realizar un análisis complementario de costo-beneficio antes de la implementación final, considerando los costos de desarrollo y despliegue, el impacto proyectado en ingresos y posibles iteraciones que permitan superar el umbral del 5%.